In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Heart_rate"

# Input paths
tcga_root_dir = "../../input/TCGA"

# Output paths
out_data_file = "../../output/preprocess/Heart_rate/TCGA.csv"
out_gene_data_file = "../../output/preprocess/Heart_rate/gene_data/TCGA.csv"
out_clinical_data_file = "../../output/preprocess/Heart_rate/clinical_data/TCGA.csv"
json_path = "../../output/preprocess/Heart_rate/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
import os
import pandas as pd

# 1. List all subdirectories in the TCGA root directory
subdirectories = os.listdir(tcga_root_dir)
print(f"Available TCGA subdirectories: {subdirectories}")

# The target trait is Heart_rate
# Define key terms relevant to heart rate
key_terms = ["heart", "cardiac", "cardiovascular", "pulse", "rhythm", "rate", "beat", "arrhythmia", 
             "tachycardia", "bradycardia", "adrenocortical"]

# Initialize variables for best match
best_match = None
best_match_score = 0
min_threshold = 1  # Require at least 1 matching term

# Convert trait to lowercase for case-insensitive matching
target_trait = trait.lower().replace("_", " ")  # "heart rate"

# Search for relevant directories
for subdir in subdirectories:
    if not os.path.isdir(os.path.join(tcga_root_dir, subdir)) or subdir.startswith('.'):
        continue
        
    subdir_lower = subdir.lower()
    
    # Check for exact matches
    if target_trait in subdir_lower:
        best_match = subdir
        print(f"Found exact match: {subdir}")
        break
    
    # Calculate score based on key terms
    score = 0
    for term in key_terms:
        if term in subdir_lower:
            score += 1
    
    # Update best match if score is higher than current best
    if score > best_match_score and score >= min_threshold:
        best_match_score = score
        best_match = subdir
        print(f"Found potential match: {subdir} (score: {score})")

# Adrenocortical cancer can affect hormones that influence heart rate
if not best_match and "TCGA_Adrenocortical_Cancer_(ACC)" in subdirectories:
    best_match = "TCGA_Adrenocortical_Cancer_(ACC)"
    print(f"Selected {best_match} as potentially relevant to heart rate studies")

# Handle the case where a match is found
if best_match:
    print(f"Selected directory: {best_match}")
    
    # 2. Get the clinical and genetic data file paths
    cohort_dir = os.path.join(tcga_root_dir, best_match)
    clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(cohort_dir)
    
    print(f"Clinical file: {os.path.basename(clinical_file_path)}")
    print(f"Genetic file: {os.path.basename(genetic_file_path)}")
    
    # 3. Load the data files
    clinical_df = pd.read_csv(clinical_file_path, sep='\t', index_col=0)
    genetic_df = pd.read_csv(genetic_file_path, sep='\t', index_col=0)
    
    # 4. Print clinical data columns for inspection
    print("\nClinical data columns:")
    print(clinical_df.columns.tolist())
    
    # Print basic information about the datasets
    print(f"\nClinical data shape: {clinical_df.shape}")
    print(f"Genetic data shape: {genetic_df.shape}")
    
    # Check if we have both gene and trait data
    is_gene_available = genetic_df.shape[0] > 0
    is_trait_available = clinical_df.shape[0] > 0
    
else:
    print(f"No suitable directory found for {trait}.")
    is_gene_available = False
    is_trait_available = False

# Record the data availability
validate_and_save_cohort_info(
    is_final=False,
    cohort="TCGA",
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Exit if no suitable directory was found
if not best_match:
    print("Skipping this trait as no suitable data was found in TCGA.")


Available TCGA subdirectories: ['TCGA_Liver_Cancer_(LIHC)', 'TCGA_Lower_Grade_Glioma_(LGG)', 'TCGA_lower_grade_glioma_and_glioblastoma_(GBMLGG)', 'TCGA_Lung_Adenocarcinoma_(LUAD)', 'TCGA_Lung_Cancer_(LUNG)', 'TCGA_Lung_Squamous_Cell_Carcinoma_(LUSC)', 'TCGA_Melanoma_(SKCM)', 'TCGA_Mesothelioma_(MESO)', 'TCGA_Ocular_melanomas_(UVM)', 'TCGA_Ovarian_Cancer_(OV)', 'TCGA_Pancreatic_Cancer_(PAAD)', 'TCGA_Pheochromocytoma_Paraganglioma_(PCPG)', 'TCGA_Prostate_Cancer_(PRAD)', 'TCGA_Rectal_Cancer_(READ)', 'TCGA_Sarcoma_(SARC)', 'TCGA_Stomach_Cancer_(STAD)', 'TCGA_Testicular_Cancer_(TGCT)', 'TCGA_Thymoma_(THYM)', 'TCGA_Thyroid_Cancer_(THCA)', 'TCGA_Uterine_Carcinosarcoma_(UCS)', '.DS_Store', 'CrawlData.ipynb', 'TCGA_Acute_Myeloid_Leukemia_(LAML)', 'TCGA_Adrenocortical_Cancer_(ACC)', 'TCGA_Bile_Duct_Cancer_(CHOL)', 'TCGA_Bladder_Cancer_(BLCA)', 'TCGA_Breast_Cancer_(BRCA)', 'TCGA_Cervical_Cancer_(CESC)', 'TCGA_Colon_and_Rectal_Cancer_(COADREAD)', 'TCGA_Colon_Cancer_(COAD)', 'TCGA_Endometrioid_Canc


Clinical data columns:
['_INTEGRATION', '_PATIENT', '_cohort', '_primary_disease', '_primary_site', 'additional_pharmaceutical_therapy', 'additional_radiation_therapy', 'age_at_initial_pathologic_diagnosis', 'atypical_mitotic_figures', 'bcr_followup_barcode', 'bcr_patient_barcode', 'bcr_sample_barcode', 'clinical_M', 'ct_scan_findings', 'cytoplasm_presence_less_than_equal_25_percent', 'days_to_birth', 'days_to_collection', 'days_to_death', 'days_to_initial_pathologic_diagnosis', 'days_to_last_followup', 'days_to_new_tumor_event_additional_surgery_procedure', 'days_to_new_tumor_event_after_initial_treatment', 'diffuse_architecture', 'distant_metastasis_anatomic_site', 'excess_adrenal_hormone_diagnosis_method_type', 'excess_adrenal_hormone_history_type', 'form_completion_date', 'gender', 'germline_testing_performed', 'histologic_disease_progression_present_indicator', 'histological_type', 'history_of_neoadjuvant_treatment', 'icd_10', 'icd_o_3_histology', 'icd_o_3_site', 'informed_consen

### Step 2: Find Candidate Demographic Features

In [3]:
# Identify candidate age and gender columns
candidate_age_cols = [
    'age_at_initial_pathologic_diagnosis', 
    'days_to_birth'
]

candidate_gender_cols = [
    'gender'
]

# Find path to the clinical file for the TCGA Adrenocortical Cancer cohort
acc_cohort_dir = os.path.join(tcga_root_dir, 'TCGA_Adrenocortical_Cancer_(ACC)')
clinical_file_path, _ = tcga_get_relevant_filepaths(acc_cohort_dir)

# Load the clinical data
clinical_df = pd.read_csv(clinical_file_path, sep='\t', index_col=0)

# Extract and preview age columns
age_preview = {}
for col in candidate_age_cols:
    if col in clinical_df.columns:
        age_preview[col] = clinical_df[col].head(5).tolist()

print("Age columns preview:")
print(age_preview)

# Extract and preview gender columns
gender_preview = {}
for col in candidate_gender_cols:
    if col in clinical_df.columns:
        gender_preview[col] = clinical_df[col].head(5).tolist()

print("\nGender columns preview:")
print(gender_preview)


Age columns preview:
{'age_at_initial_pathologic_diagnosis': [58, 44, 23, 23, 30], 'days_to_birth': [-21496, -16090, -8624, -8451, -11171]}

Gender columns preview:
{'gender': ['MALE', 'FEMALE', 'FEMALE', 'FEMALE', 'MALE']}


### Step 3: Select Demographic Features

In [4]:
# Examine the age columns to select the most appropriate one
age_col = None
gender_col = None

# Age column inspection
age_columns_preview = {'age_at_initial_pathologic_diagnosis': [58, 44, 23, 23, 30], 
                       'days_to_birth': [-21496, -16090, -8624, -8451, -11171]}

if age_columns_preview:
    # Both columns seem to have valid data
    # 'age_at_initial_pathologic_diagnosis' is directly in years and easier to interpret
    # 'days_to_birth' is in negative days (days before current date) which needs conversion
    age_col = 'age_at_initial_pathologic_diagnosis'

# Gender column inspection
gender_columns_preview = {'gender': ['MALE', 'FEMALE', 'FEMALE', 'FEMALE', 'MALE']}

if gender_columns_preview:
    # The gender column contains valid information
    gender_col = 'gender'

# Print the chosen columns
print(f"Selected age column: {age_col}")
print(f"Selected gender column: {gender_col}")


Selected age column: age_at_initial_pathologic_diagnosis
Selected gender column: gender


### Step 4: Feature Engineering and Validation

In [5]:
# Step 1: Extract and standardize clinical features
# Create clinical features dataframe with trait (Canavan Disease) using patient IDs
clinical_features = tcga_select_clinical_features(
    clinical_df, 
    trait=trait, 
    age_col=age_col, 
    gender_col=gender_col
)

# Step 2: Normalize gene symbols in the gene expression data
# The gene symbols in TCGA genetic data are already standardized, but we'll normalize them for consistency
normalized_gene_df = normalize_gene_symbols_in_index(genetic_df)

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_df.to_csv(out_gene_data_file)
print(f"Normalized gene expression data saved to {out_gene_data_file}")
print(f"Gene expression data shape after normalization: {normalized_gene_df.shape}")

# Step 3: Link clinical and genetic data
# Transpose genetic data to have samples as rows and genes as columns
genetic_df_t = normalized_gene_df.T
# Save the clinical data for reference
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_features.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")
print(f"Clinical data shape: {clinical_features.shape}")

# Verify common indices between clinical and genetic data
clinical_indices = set(clinical_features.index)
genetic_indices = set(genetic_df_t.index)
common_indices = clinical_indices.intersection(genetic_indices)
print(f"Number of samples in clinical data: {len(clinical_indices)}")
print(f"Number of samples in genetic data: {len(genetic_indices)}")
print(f"Number of common samples: {len(common_indices)}")

# Link the data by using the common indices
linked_data = pd.concat([clinical_features.loc[list(common_indices)], genetic_df_t.loc[list(common_indices)]], axis=1)
print(f"Linked data shape: {linked_data.shape}")

# Step 4: Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait_col=trait)
print(f"Data shape after handling missing values: {linked_data.shape}")

# Step 5: Determine whether the trait and demographic features are severely biased
trait_biased, linked_data = judge_and_remove_biased_features(linked_data, trait=trait)

# Step 6: Conduct final quality validation and save information
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort="TCGA",
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=trait_biased,
    df=linked_data,
    note=f"Dataset contains TCGA glioma and brain tumor samples with gene expression and clinical information for {trait}."
)

# Step 7: Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset deemed not usable based on validation criteria. Data not saved.")

print("Preprocessing completed.")

Normalized gene expression data saved to ../../output/preprocess/Heart_rate/gene_data/TCGA.csv
Gene expression data shape after normalization: (19848, 79)
Clinical data saved to ../../output/preprocess/Heart_rate/clinical_data/TCGA.csv
Clinical data shape: (92, 3)
Number of samples in clinical data: 92
Number of samples in genetic data: 79
Number of common samples: 79
Linked data shape: (79, 19851)


Data shape after handling missing values: (79, 19851)
Quartiles for 'Heart_rate':
  25%: 1.0
  50% (Median): 1.0
  75%: 1.0
Min: 1
Max: 1
The distribution of the feature 'Heart_rate' in this dataset is severely biased.

Quartiles for 'Age':
  25%: 35.0
  50% (Median): 49.0
  75%: 59.5
Min: 14
Max: 77
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1' with 31 occurrences. This represents 39.24% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

Dataset deemed not usable based on validation criteria. Data not saved.
Preprocessing completed.
